In [1]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects

In [2]:
df=pd.read_excel("bankfull.ods", engine="odf")

In [3]:
df.head(1000)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,32,entrepreneur,married,basic.6y,no,yes,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
996,41,services,single,high.school,no,yes,yes,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
997,59,housemaid,married,basic.6y,no,yes,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
998,57,technician,married,basic.9y,no,yes,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no


In [10]:
df.isnull().values.any()

False

In [11]:
x_cols = [c for c in df.columns if c != 'y']

In [12]:
x_cols

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

In [13]:
X = df[x_cols]

In [14]:
y=df['y']

In [15]:
y

0         no
1         no
2         no
3         no
4         no
        ... 
41183    yes
41184     no
41185     no
41186    yes
41187     no
Name: y, Length: 41188, dtype: object

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1234)

In [17]:
train_mode = dict(X_train.mode().iloc[0])
#X_train = X_train.fillna(train_mode)
print(train_mode)

{'age': 31, 'job': 'admin.', 'marital': 'married', 'education': 'university.degree', 'default': 'no', 'housing': 'yes', 'loan': 'no', 'contact': 'cellular', 'month': 'may', 'day_of_week': 'thu', 'duration': 90, 'campaign': 1, 'pdays': 999, 'previous': 0, 'poutcome': 'nonexistent', 'emp.var.rate': 1.4, 'cons.price.idx': 93.994, 'cons.conf.idx': -36.4, 'euribor3m': 4.857, 'nr.employed': 5228.1}


In [18]:
encoders = {}
for column in ['education', 'marital','job',
                'month', 'housing', 'day_of_week','poutcome', 'loan', 'contact', 'default']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

<ipython-input-18-f18b3e341ad6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[column] = categorical_convert.fit_transform(X_train[column])


In [19]:
rf = RandomForestClassifier(n_estimators = 100,verbose=1)
rf = rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished


In [20]:
rf.predict_proba(X_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


array([[1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.96, 0.04],
       [1.  , 0.  ],
       [1.  , 0.  ]])

In [33]:
rf

RandomForestClassifier(verbose=1)

In [34]:
et = ExtraTreesClassifier(n_estimators = 100)
et = et.fit(X_train, y_train)

In [35]:
joblib.dump(train_mode, "./train_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)

['./extra_trees.joblib']

In [36]:
X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
31305,31,1,1,1,1,2,0,0,6,4,148,2,999,0,1,-1.8,92.893,-46.2,1.334,5099.1
24892,36,0,1,3,0,0,0,0,7,3,141,2,999,0,1,-0.1,93.200,-42.0,4.153,5195.8
1986,36,1,2,3,0,0,2,1,6,1,163,2,999,0,1,1.1,93.994,-36.4,4.857,5191.0
8346,58,9,1,6,0,2,0,1,4,3,108,12,999,0,1,1.4,94.465,-41.8,4.864,5228.1
29116,33,9,1,5,0,0,2,0,0,0,404,1,999,1,0,-1.8,93.075,-47.1,1.405,5099.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32399,54,0,0,5,0,2,0,1,6,0,75,2,999,0,1,-1.8,92.893,-46.2,1.313,5099.1
17048,60,5,1,3,1,0,0,1,3,0,133,7,999,0,1,1.4,93.918,-42.7,4.962,5228.1
23924,43,9,1,3,0,2,0,1,1,0,251,3,999,0,1,1.4,93.444,-36.1,4.963,5228.1
34086,39,4,0,6,0,0,0,0,6,4,157,4,999,0,1,-1.8,92.893,-46.2,1.281,5099.1
